In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
from sklearn.neighbors import NearestNeighbors
from pathlib import Path 

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
def find_min_round(array):
    array = np.array(array)
    x = round(np.min(array[np.nonzero(array)]))
    return(x)

def find_max_round(array):
    array = np.array(array)
    x = round(np.max(array))
    return(x)

In [ ]:
def filtering(skeletons):
    """if value is missing or low proba then it returns True"""
    temp_list = []
    for key,value in skeletons.items():
        x_coord = value[0][1]
        proba = value[2][1]
        if x_coord ==0 or proba<0.4:
            temp_list.append(True)
        else:
            temp_list.append(False)
    return(np.any(temp_list))

In [ ]:
def nearest_neighbour(new_skeleton, various_skeletons):
    joint=1
    neigh.fit([[new_skeleton[-1][0][joint],  new_skeleton[-1][1][joint]]])
    if len(various_skeletons) ==1:
        dist, nn = neigh.kneighbors([[list(various_skeletons.values())[0][0][joint], list(various_skeletons.values())[0][1][joint]]], 
                                        return_distance = True)
        return([dist])
    elif len(various_skeletons)==2:
        dist1, nn = neigh.kneighbors([[list(various_skeletons.values())[0][0][joint], list(various_skeletons.values())[0][1][joint]]], 
                                        return_distance = True)
        dist2, nn = neigh.kneighbors([[list(various_skeletons.values())[1][0][joint], list(various_skeletons.values())[1][1][joint]]], 
                                        return_distance = True)
        return([dist1, dist2])
    elif len(various_skeletons) ==3:
        dist1, nn = neigh.kneighbors([[list(various_skeletons.values())[0][0][joint], list(various_skeletons.values())[0][1][joint]]], 
                                        return_distance = True)
        dist2, nn = neigh.kneighbors([[list(various_skeletons.values())[1][0][joint], list(various_skeletons.values())[1][1][joint]]], 
                                        return_distance = True)
        dist3, nn = neigh.kneighbors([[list(various_skeletons.values())[2][0][joint], list(various_skeletons.values())[2][1][joint]]], 
                                        return_distance = True)
        return([dist1,dist2,dist3])

In [ ]:
def max_proba(skeleton):
    prob_list = []
    keys = list(skeleton.keys())
    values = list(skeleton.values())
    if len(skeleton) ==1:
        return(keys[0],values[0])
    elif len(skeleton) ==2:
        for key in skeleton.keys():
            prob_list.append(np.mean(skeleton[key][2]))
        return(keys[np.argmax(prob_list)],values[np.argmax(prob_list)])
           
    elif len(skeleton) ==3:
        for key in skeleton.keys():
            prob_list.append(np.mean(skeleton[key][2]))
        return(keys[np.argmax(prob_list)],values[np.argmax(prob_list)])
    
    

In [ ]:
skeletons[74]['skeleton2'][0][0] ==0

In [ ]:
list(skeletons[74].values())[0][2]

In [ ]:
np.mean(skeletons[74]['skeleton2'][2])

In [ ]:
openpose_path = '/Users/andreibirladeanu/Documents/Data/meal_openpose/1047_meal/'

In [ ]:
coords = [file for file in sorted_alphanumeric(os.listdir(openpose_path)) if file[0] !="."]


In [ ]:
skeletons = []
frames = []
for frame in coords:
    data = pd.read_csv(openpose_path + frame)
    data = data.fillna(0)
    if (len(data.columns)-1)/3 == 1:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']), np.array(data['prob_1'])]})
        frames.append(int(str(frame.split('.')[0]).split("_")[1]))
    elif (len(data.columns)-1)/3 == 2:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                                                                                                                         np.array(data['prob_2'])]})
        frames.append(int(str(frame.split('.')[0]).split("_")[1]))
    elif (len(data.columns)-1)/3 == 3:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                         np.array(data['prob_2'])],
                      'skeleton3':[np.array(data['x_3']), np.array(data['y_3']),  np.array(data['prob_3'])]}) 
        frames.append(int(str(frame.split('.')[0]).split("_")[1]))

In [ ]:
for 

In [ ]:
### extracting skel 1
joint = 1
thr = 150
neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton1 = []
frame_no = []
for i, frm in zip(range(len(skeletons)), frames): 
    if filtering(skeletons[i])==True: ### checks if the joint was detected in all the skeletons and probability is high
        continue
    elif new_skeleton1 == []:
        new_skeleton1.append(max_proba(skeletons[i])[1]
        skeletons[i].pop((max_proba(skeletons[i])[0])
        frame_no.append(frm)
    elif len(skeletons[i])==1:
        dist = nearest_neighbour(new_skeleton1, skeletons[i])
        if dist[0][0][0] < thr:
            new_skeleton1.append(list(skeletons[i].values())[0])
            frame_no.append(frm)
            skeletons[i].pop(list(skeletons[i].keys())[0])
        else:
            continue                  
    elif len(skeletons[i])==2:
        dist = nearest_neighbour(new_skeleton1, skeletons[i])
        distance_1 = dist[0][0][0]; distance_2 = dist[1][0][0]
        if distance_1 < distance_2 and distance_1< thr:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton1') 
        elif distance_1 > distance_2 and distance_2<thr:
            new_skeleton1.append(skeletons[i]['skeleton2'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton2')  
        else:
            continue
    elif len(skeletons[i])==3:
        dist = nearest_neighbour(new_skeleton1, skeletons[i])
        distance_1 = dist[0][0][0]; distance_2 = dist[1][0][0]; distance_3 = dist[2][0][0]
        if distance_1 < distance_2 < distance_3 and distance_1 < thr:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton1')  
        elif distance_2 < distance_1 < distance_3 and distance_2 < thr:
            new_skeleton1.append(skeletons[i]['skeleton2'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton2')
        elif distance_3 < distance_1 < distance_2 and distance_3 < thr:
            new_skeleton1.append(skeletons[i]['skeleton3'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton3')
        else:
            continue
for skeleton, f_no in zip(new_skeleton1, frame_no):
    out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
    filepath = Path('/Users/andreibirladeanu/Documents/Data/1047_meal_openpose/' +'skel1/' +str(f_no) +'.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    out.to_csv(filepath)

print('done')
    

In [ ]:
### second pass skeleton2:
thr = 80
neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton2 = []
frame_no = []
for i, frm in zip(range(len(skeletons)), frames): 
    if skeletons[i] == {}:
        continue
    elif filtering(skeletons[i])==True: ### checks if the joint was detected in all the skeletons
        continue
    elif new_skeleton2 == []:
        new_skeleton2.append(list(skeletons[i].values())[0])
        skeletons[i].pop(list(skeletons[i].keys())[0])
        frame_no.append(frm)
    elif len(skeletons[i])==1:
        dist = nearest_neighbour(new_skeleton2, skeletons[i])
        distance = dist[0][0][0]
        if distance < thr:
            new_skeleton2.append(list(skeletons[i].values())[0])
            frame_no.append(frm)
            skeletons[i].pop(list(skeletons[i].keys())[0])
        else:
            continue                  
    elif len(skeletons[i]) ==2:
        dist = nearest_neighbour(new_skeleton2, skeletons[i])
        distance_1 = dist[0][0][0]; distance_2 = dist[1][0][0]
        if distance_1 < distance_2 and distance_1< thr:
            new_skeleton2.append(list(skeletons[i].values())[0])
            frame_no.append(frm)
            skeletons[i].pop(list(skeletons[i].keys())[0])
        elif distance_1 > distance_2 and distance_2<thr:
            new_skeleton2.append(list(skeletons[i].values())[1])
            frame_no.append(frm)
            skeletons[i].pop(list(skeletons[i].keys())[1])
        else:
            continue
for skeleton, f_no in zip(new_skeleton2, frame_no):
    out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
    filepath = Path('/Users/andreibirladeanu/Documents/Data/1047_meal_openpose/' +'skel2/' +str(f_no) +'.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    out.to_csv(filepath)

In [ ]:
skeletons[0]

In [ ]:
neigh.fit([[5, 10]])
dist, nn = neigh.kneighbors([[5,10]])
print(dist)

In [ ]:
#### third pass
new_skeleton3 = []
frame_no = []
for i, frm in zip(range(len(skeletons)), frames): 
    if skeletons[i] == {}:
        continue
    elif check_missing(skeletons[i])==True: ### checks if the joint was detected in all the skeletons
        continue
    if len(skeletons[i]) != {}:
        new_skeleton3.append(list(skeletons[i].items())[0][1])
        frame_no.append(frm)
for skeleton, f_no in zip(new_skeleton3, frame_no):
    out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
    filepath = Path('/Users/andreibirladeanu/Documents/Data/1047_meal_openpose/' +'skel3/' +str(f_no) +'.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    out.to_csv(filepath)


In [ ]:
print('a')

In [ ]:
skeletons[1]['skeleton1'][1]

In [ ]:
check_missing(skeletons[20])

In [ ]:
######### sample 5 videos

In [ ]:
video_path = '/Users/andreibirladeanu/Documents/Data/meal_videos/'
videos = [x for x in sorted_alphanumeric(os.listdir(video_path)) if x[0]!='.']

In [ ]:
openpose_path = '/Users/andreibirladeanu/Documents/Data/meal_openpose/'
coords = [x for x in sorted_alphanumeric(os.listdir(openpose_path)) if x[0]!='.']

In [ ]:
coords

In [ ]:
for coord in tqdm(coords):
    new_path = openpose_path+coord
    os.chdir(new_path)
    skeletons = []
    frames = []
    for frame in  sorted_alphanumeric(os.listdir(new_path))[1:]:
        data = pd.read_csv(openpose_path + coord+'/'+frame)
        data = data.fillna(0)
        if (len(data.columns)-1)/3 == 1:
            skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']), np.array(data['prob_1'])]})
            frames.append(int(str(frame.split('.')[0]).split("_")[1]))
        elif (len(data.columns)-1)/3 == 2:
            skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                                                                                                                             np.array(data['prob_2'])]})
            frames.append(int(str(frame.split('.')[0]).split("_")[1]))
        elif (len(data.columns)-1)/3 == 3:
            skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                             np.array(data['prob_2'])],
                          'skeleton3':[np.array(data['x_3']), np.array(data['y_3']),  np.array(data['prob_3'])]}) 
            frames.append(int(str(frame.split('.')[0]).split("_")[1]))
    ### first pass 
    ### extracting skel 1
    joint = 1
    thr = 150
    neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
    new_skeleton1 = []
    frame_no = []
    for i, frm in zip(range(len(skeletons)), frames): 
        if filtering(skeletons[i])==True: ### checks if the joint was detected in all the skeletons and probability is high
            continue
        elif new_skeleton1 == []:
            new_skeleton1.append(max_proba(skeletons[i])[1])
            skeletons[i].pop(max_proba(skeletons[i])[0])
            frame_no.append(frm)
        elif len(skeletons[i])==1:
            dist = nearest_neighbour(new_skeleton1, skeletons[i])
            if dist[0][0][0] < thr:
                new_skeleton1.append(list(skeletons[i].values())[0])
                frame_no.append(frm)
                skeletons[i].pop(list(skeletons[i].keys())[0])
            else:
                continue                  
        elif len(skeletons[i])==2:
            dist = nearest_neighbour(new_skeleton1, skeletons[i])
            distance_1 = dist[0][0][0]; distance_2 = dist[1][0][0]
            if distance_1 < distance_2 and distance_1< thr:
                new_skeleton1.append(list(skeletons[i].values())[0])
                frame_no.append(frm)
                skeletons[i].pop(list(skeletons[i].keys())[0])
            elif distance_1 > distance_2 and distance_2<thr:
                new_skeleton1.append(list(skeletons[i].values())[1])
                frame_no.append(frm)
                skeletons[i].pop(list(skeletons[i].keys())[1])
            else:
                continue
        elif len(skeletons[i])==3:
            dist = nearest_neighbour(new_skeleton1, skeletons[i])
            distance_1 = dist[0][0][0]; distance_2 = dist[1][0][0]; distance_3 = dist[2][0][0]
            if distance_1 < distance_2 < distance_3 and distance_1 < thr:
                new_skeleton1.append(list(skeletons[i].values())[0])
                frame_no.append(frm)
                skeletons[i].pop(list(skeletons[i].keys())[0]) 
            elif distance_2 < distance_1 < distance_3 and distance_2 < thr:
                new_skeleton1.append(list(skeletons[i].values())[1])
                frame_no.append(frm)
                skeletons[i].pop(list(skeletons[i].keys())[1])
            elif distance_3 < distance_1 < distance_2 and distance_3 < thr:
                new_skeleton1.append(list(skeletons[i].values())[2])
                frame_no.append(frm)
                skeletons[i].pop(list(skeletons[i].keys())[2])
            else:
                continue
    for skeleton, f_no in zip(new_skeleton1, frame_no):
        out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
        filepath = Path('/Users/andreibirladeanu/Documents/Data/meal_tracked/'+coord +'/skel1/' +str(f_no) +'.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        out.to_csv(filepath)

        
    ### second pass
    thr = 80
    neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
    new_skeleton2 = []
    frame_no = []
    for i, frm in zip(range(len(skeletons)), frames): 
        if skeletons[i] == {}:
            continue
        elif filtering(skeletons[i])==True: ### checks if the joint was detected in all the skeletons
            continue
        elif new_skeleton2 == []:
            new_skeleton2.append(list(skeletons[i].values())[0])
            skeletons[i].pop(list(skeletons[i].keys())[0])
            frame_no.append(frm)
        elif len(skeletons[i])==1:
            dist = nearest_neighbour(new_skeleton2, skeletons[i])
            distance = dist[0][0][0]
            if distance < thr:
                new_skeleton2.append(list(skeletons[i].values())[0])
                frame_no.append(frm)
                skeletons[i].pop(list(skeletons[i].keys())[0])
            else:
                continue                  
        elif len(skeletons[i]) ==2:
            dist = nearest_neighbour(new_skeleton2, skeletons[i])
            distance_1 = dist[0][0][0]; distance_2 = dist[1][0][0]
            if distance_1 < distance_2 and distance_1< thr:
                new_skeleton2.append(list(skeletons[i].values())[0])
                frame_no.append(frm)
                skeletons[i].pop(list(skeletons[i].keys())[0])
            elif distance_1 > distance_2 and distance_2<thr:
                new_skeleton2.append(list(skeletons[i].values())[1])
                frame_no.append(frm)
                skeletons[i].pop(list(skeletons[i].keys())[1])
            else:
                continue
    for skeleton, f_no in zip(new_skeleton2, frame_no):
        out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
        filepath = Path('/Users/andreibirladeanu/Documents/Data/meal_tracked/'+coord +'/skel2/' +str(f_no) +'.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        out.to_csv(filepath)
        
        
    ### third pass 
    new_skeleton3 = []
    frame_no = []
    for i, frm in zip(range(len(skeletons)), frames): 
        if skeletons[i] == {}:
            continue
        elif filtering(skeletons[i])==True: ### checks if the joint was detected in all the skeletons
            continue
        if len(skeletons[i]) != {}:
            head = list(skeletons[i].values())[0][0][0]
            avg_prob = np.mean(list(skeletons[i].values())[0][2])
        ### if the head is missing and overall probabilities less than 50%
            if head ==0 and avg_prob <0.5:
                continue
            else:
                new_skeleton3.append(list(skeletons[i].items())[0][1])
                frame_no.append(frm)
    for skeleton, f_no in zip(new_skeleton3, frame_no):
        out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
        filepath = Path('/Users/andreibirladeanu/Documents/Data/meal_tracked/'+coord+ '/skel3/' +str(f_no) +'.csv')
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        out.to_csv(filepath)

In [ ]:
import random
path_tracked = '/Users/andreibirladeanu/Documents/Data/experimenting/tracked/'
tracked = [x for x in sorted_alphanumeric(os.listdir(path_tracked)) if x[0]!='.']
path_video = '/Users/andreibirladeanu/Documents/Data/experimenting/videos/'
skeleton = []
skel = '/skel1/'
skeletons = []
for pose, video in zip(tracked, coords):
    temp_list = []
    for skelet in random.sample(sorted_alphanumeric(os.listdir(path_tracked+ pose+skel)), k=100):
            temp_list.append(skelet)
    skeletons.append(temp_list)

In [ ]:
for skelet, video in zip(skeletons, videos):
    cap = cv2.VideoCapture(path_video+video)
    new_folder = '/Users/andreibirladeanu/Documents/Data/experimenting/visualize/'+ video.split(".")[0]+skel
    os.makedirs(new_folder, exist_ok = True)
    os.chdir(new_folder)
    for frame in skelet:
        data = pd.read_csv(path_tracked + video.split(".")[0] + skel + frame)
        cap.set(1,int(frame.split('.')[0]))
        ret, image = cap.read()
        cv2.imwrite("frame" + str(frame.split('.')[0]) + '.png', image[find_min_round(data['y']):find_max_round(data['y']),
                       find_min_round(data['x']):find_max_round(data['x'])])


In [ ]:
new_folder